In [1]:
import sys
sys.path.append("../")
import pandas as pd
import plotly.graph_objects as go
import datetime as dt
from plotting import CandlePlot

In [2]:
from infrastructure.instrument_collection import instrumentCollection as ic

In [3]:
pair = "AUD_CAD"
granularity = "H4"
df = pd.read_pickle(f"../data/{pair}_{granularity}.pkl")
MA_LIST = [10, 20, 50, 100, 200]

In [4]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [5]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c
0,2021-11-08 22:00:00+00:00,0.92370,0.92420,0.92121,0.92133
1,2021-11-09 02:00:00+00:00,0.92132,0.92337,0.92032,0.92294
2,2021-11-09 06:00:00+00:00,0.92294,0.92419,0.92246,0.92350
3,2021-11-09 10:00:00+00:00,0.92354,0.92379,0.92030,0.92132
4,2021-11-09 14:00:00+00:00,0.92134,0.92240,0.91718,0.91766


In [6]:
for ma in MA_LIST:    
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(inplace=True, drop=True)

In [7]:
df_plot = df_ma.iloc[:500]

In [8]:
cp = CandlePlot(df_plot)

In [9]:
traces = [ f"MA_{x}" for x in MA_LIST ]

In [ ]:
cp.show_plot(line_traces=traces)

In [10]:
MA_S = "MA_50"
MA_L = "MA_200"
BUY = 1
SELL = -1
NONE = 0

In [11]:
df_an = df_ma[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', MA_S, MA_L]].copy()

In [12]:
df_an.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200
0,2021-12-24 02:00:00+00:00,0.92727,0.92828,0.92640,0.92772,0.920381,0.914325
1,2021-12-24 06:00:00+00:00,0.92772,0.92883,0.92726,0.92792,0.920707,0.914358
2,2021-12-24 10:00:00+00:00,0.92792,0.92810,0.92532,0.92688,0.921031,0.914378
3,2021-12-24 14:00:00+00:00,0.92687,0.92741,0.92505,0.92552,0.921285,0.914388
4,2021-12-24 18:00:00+00:00,0.92558,0.92793,0.92465,0.92513,0.921529,0.914407


In [13]:
df_an['DELTA'] = df_an.MA_50 - df_an.MA_200
df_an['DELTA_PREV'] = df_an.DELTA.shift(1)

In [14]:
def is_trade(row):
    if row.DELTA >= 0  and row.DELTA_PREV < 0:
        return BUY
    elif row.DELTA < 0  and row.DELTA_PREV >= 0:
        return SELL
    return NONE

In [15]:
df_an['TRADE'] = df_an.apply(is_trade, axis=1)

In [16]:
df_trades = df_an[df_an.TRADE != NONE].copy()

In [17]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200,DELTA,DELTA_PREV,TRADE
77,2022-01-11 22:00:00+00:00,0.90688,0.90716,0.90560,0.90602,0.915155,0.915196,-0.000041,3.633000e-04,-1
225,2022-02-15 14:00:00+00:00,0.90900,0.91154,0.90830,0.91066,0.906913,0.906897,0.000015,-2.038500e-04,1
496,2022-04-19 17:00:00+00:00,0.93124,0.93168,0.93020,0.93108,0.935758,0.935859,-0.000101,1.091000e-04,-1
697,2022-06-06 05:00:00+00:00,0.90583,0.90814,0.90542,0.90807,0.909366,0.909278,0.000088,-1.000000e-06,1
719,2022-06-09 21:00:00+00:00,0.90124,0.90196,0.90050,0.90150,0.906860,0.906952,-0.000091,2.965000e-05,-1


In [18]:
cp = CandlePlot(df_an.iloc[220:260])
cp.show_plot(line_traces=[MA_S, MA_L])

In [ ]:
ic.LoadInstruments("../data")

In [ ]:
ic.instruments_dict[pair]

In [ ]:
ins_data = ic.instruments_dict[pair]

In [ ]:
df_trades.shape

In [ ]:
df_trades['DIFF'] = df_trades.mid_c.diff().shift(-1)
df_trades.fillna(0, inplace=True)

In [ ]:
df_trades['GAIN'] = df_trades['DIFF'] / ins_data.pipLocation
df_trades['GAIN'] = df_trades['GAIN'] * df_trades['TRADE'] 

In [ ]:
df_trades.GAIN.sum()

In [ ]:
df_trades['GAIN_C'] = df_trades['GAIN'].cumsum()

In [ ]:
cp = CandlePlot(df_trades, candles=False)
cp.show_plot(line_traces=['GAIN_C'])